# Testing code

# Importing stuff

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings

warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
import os
import os.path as osp
import random

import mlflow
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm_notebook

from nlpclass.config import model_config
from nlpclass.data.data_utils import TranslationDataset, text_collate_func
from nlpclass.models.evaluation_utils import bleu_eval, output_to_translations
from nlpclass.models.models import DecoderRNN, EncoderRNN, TranslationModel
from nlpclass.models.training_utils import load_data

In [3]:
CURRENT_PATH = os.getcwd()
DATA_DIR = osp.join(CURRENT_PATH, '..', 'data')
MODEL_DIR = osp.join(CURRENT_PATH, '..','models')

# Data

In [4]:
data, data_loaders, max_length = load_data('vi', batch_size=24, subsample=0.25)

Counting words...
Counted words:
eng 17735
vi 25085
Counting words...
Counted words:
eng 3039
vi 3678
Counting words...
Counted words:
eng 2890
vi 3518


In [20]:
encoder = EncoderRNN(
    data['train'].input_lang.n_words,
    embedding_size=100,
    hidden_size=128,
    num_layers=1,
    dropout=0.0,
    bidirectional=False).to(model_config.device)
if encoder.bidirectional:
    multiplier = 2
else:
    multiplier = 1
decoder = DecoderRNN(
    embedding_size=100,
    hidden_size=multiplier * 128,
    output_size=data['train'].output_lang.n_words,
    attention=True).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=0.5).to(model_config.device)

In [21]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)

In [22]:
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [23]:
def calc_loss(logits, target, criterion):
    logits_flat = logits.view(-1, logits.size(-1))
    target_flat = target.view(-1, 1).squeeze()
    return criterion(logits_flat, target_flat)

In [ ]:
for i in tqdm_notebook(range(25)):
    for batch in data_loaders['train']:
        optimizer.zero_grad()
        logits = translation_model(batch)
        loss = calc_loss(logits, batch['target'], criterion)
        loss.backward()
        
        encoder_norm = 0
        for p in translation_model.encoder.parameters():
            param_norm = p.grad.data.norm(2)
            encoder_norm += param_norm.item() ** 2
        decoder_norm = 0
        for p in translation_model.decoder.parameters():
            param_norm = p.grad.data.norm(2)
            decoder_norm += param_norm.item() ** 2
            
        print(encoder_norm, decoder_norm, loss.item())
            
        clip_grad_norm_(filter(lambda p: p.requires_grad,
                                   translation_model.parameters()), 5.0)
        
        optimizer.step() 
    original, translation = evaluate(translation_model, data, data_loaders, dataset_type='train')

0.0007507604458805816 6.738299269789271 10.488754272460938
0.007891493239795301 0.5306020186921313 10.093649864196777
0.011306982743482009 1.2984497982324668 10.039058685302734
0.023662760823424375 2.7244228574921108 9.905583381652832
0.2613200514727498 42.728482684429 9.854333877563477
0.09364816426852679 15.203862072483052 9.403263092041016
0.004118706752563063 6.754813569752274 9.004295349121094
0.0016209569427997445 8.610039973229759 8.606600761413574
0.00163209223259148 6.060230612222734 8.490111351013184
0.0011296350984670695 5.144811316260217 8.137129783630371
0.0014982526160880103 5.204859946310725 7.672852993011475
0.0011092511318612037 3.9895927082342215 7.902891159057617
0.001741556854446898 4.104527079466306 7.678654670715332
0.0031986945226238762 2.42971230241077 7.369710922241211
0.003401476478850956 2.179890618937002 7.533790588378906
0.004310077971122467 1.709922460185091 7.622716426849365
0.004998833243576786 3.5907971461558734 7.322064399719238
0.0036611069605819433 2

0.00013830177517062755 1.7742978266125082 6.640829563140869
0.00017562025796837624 1.3546848492600148 6.495552062988281
7.745027929405603e-05 1.683356977393802 6.463523864746094
8.041722864114908e-05 1.5709942767583172 6.708235263824463
8.509464650451955e-05 2.0779902876365997 6.978182792663574
6.305309287741474e-05 1.4590393037509306 6.896634101867676
0.0001905756384047232 1.9012411025769866 6.588395118713379
6.351431804534351e-05 1.8067135302384612 6.793454170227051
7.677566794735645e-05 1.4138626283159372 6.811461448669434
0.00011502792108024293 1.4741482679668556 6.496173858642578
6.876097531786159e-05 1.092171212202331 6.8133625984191895
0.00021309307103379986 1.9541082606133786 6.864954948425293
0.00012779492316566398 1.353237043298314 6.533205509185791
0.00013210139735011175 1.5333709794024115 6.736357688903809
8.241344872619369e-05 1.30853159515686 6.965826511383057
6.00032794554225e-05 1.4022543123308309 6.596604347229004
5.758312175810071e-05 1.9403898575482694 6.734641551971

In [13]:
x = next(iter(data_loaders['train']))
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.greedy(x), data['train'])
bleu_eval(original, translations)

0.024467507482357514

In [16]:
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.beam_search(x), data['train'])
bleu_eval(original, translations)

0.0023789236679995717

In [24]:
x = next(iter(data_loaders['train']))
input_seq = x['input']
target_seq = x['target']
input_length = x['input_length']
target_length = x['target_length']

encoded_input, hidden = translation_model.encoder.forward(input_seq, input_length)

In [25]:
encoded_input[0]

tensor([[-0.6125, -0.5094,  0.1136,  ...,  0.2936, -0.0332, -0.5566],
        [-0.2966, -0.5157,  0.0630,  ...,  0.6734,  0.0263, -0.9648],
        [ 0.3257, -1.1140,  0.3586,  ...,  0.7310, -0.7097, -0.5748],
        ...,
        [-0.3053, -0.1229,  0.2143,  ..., -0.5936, -0.5418, -0.0655],
        [-0.2868, -0.2641,  0.6324,  ..., -0.5235, -0.4746, -0.1487],
        [-0.0230, -0.0655,  0.1169,  ..., -0.4210,  0.3063,  0.4462]],
       device='cuda:0', grad_fn=<SelectBackward>)

In [26]:
x = next(iter(data_loaders['train']))

optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-2)

for i in range(100):
    optimizer.zero_grad()
    logits = translation_model(x)
    loss = calc_loss(logits, x['target'], criterion)
    print(loss)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(10.8560, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.9659, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.5004, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.8864, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.6769, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.1789, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.1030, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.1107, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.3734, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.3272, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.8524, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.8219, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.5111, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.5380, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.2955, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.0173, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1326, device='cuda:0', grad_fn=<NllLossBackwar

In [28]:
def evaluate(model, data, data_loaders, dataset_type='dev', max_batches=100):
    model.eval()
    with torch.no_grad():
        original_strings = []
        translated_strings = []
        for i, batch in enumerate(data_loaders[dataset_type]):
            if i > max_batches:
                break
            logits = translation_model(batch)
            epoch_loss = calc_loss(logits, batch['target'], criterion)
            original = output_to_translations(batch['target'], data['train'])
            translations = output_to_translations(model.greedy(batch), data['train'])
            original_strings.extend(original)
            translated_strings.extend(translations)
        bleu = bleu_eval(original_strings, translated_strings)
        model.train()
        print(epoch_loss)
        print(bleu)
        
        return original_strings, translated_strings

In [29]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [30]:
for i, batch in enumerate(tqdm_notebook(data_loaders['train'])):
    if i % 500 == 0:
        evaluate(translation_model, data, data_loaders)
        #evaluate(translation_model, data, data_loaders, dataset_type='train')
    optimizer.zero_grad()
    logits = translation_model(batch)
    loss = calc_loss(logits, batch['target'], criterion)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(19.1450, device='cuda:0')
0.0008142795832761162
tensor(9.0464, device='cuda:0')
0.0033784249080402885


KeyboardInterrupt: 

In [ ]:
original_strings, translated_strings = evaluate(translation_model, data, data_loaders)

In [13]:
original_strings[0:10]

['baseball be later but of volumes all. hope. answer. only were',
 'be this. 65. know what must truth. i most',
 'games be practical everybody pins your needles all forward careful were',
 'i moment. it attack. be sorrows. ten.',
 'canadian be appointment careful laugh. no truth. something time were',
 'beautiful cupboard. hot. wine these',
 'much. but how bitter really dishonesty fed recently',
 'applied all else fed to climate husbands. matter. key',
 'but they answer. perfect. today. tape',
 'i is invented i situation getting powers your few']

In [14]:
translated_strings[0:10]

['i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the']

In [48]:
decoder_input.size()

torch.Size([16])

In [23]:
total_loss.backward()

In [30]:
predictions = translation_model.greedy(x)

In [215]:
for row in predictions.cpu().numpy():
    decoded_words = []
    for elem in row[1:]:
        decoded_words.append(data['train']['output_lang'].index2word[elem])
        if elem == model_config.EOS_token:
            break

In [167]:
yo = Variable(torch.LongTensor([model_config.SOS_token] * 8)).to(model_config.device)
yo = torch.stack((yo, topi.squeeze(), topi.squeeze()), dim=1)

In [99]:
total_loss.backward()

In [57]:
seq_range = torch.autograd.Variable(torch.LongTensor(np.repeat([2], len(x['input_length'])))).to(model_config.device)
mask = seq_range < x['input_length']
loss = -torch.gather(decoder_output, dim=1, index=input_var.unsqueeze(1)).squeeze() * mask.float()

In [66]:
loss.sum() / torch.sum(loss > 0).float()

tensor(11.4193, device='cuda:0', grad_fn=<DivBackward1>)

In [63]:
torch.sum(loss > 0).cpu().numpy()

array(8)

In [ ]:
encoder_output, encoder_hidden = encoder(x['input'], x['input_length'])

In [290]:
context = None
if decoder.attention:
    context = Variable(torch.zeros(encoder_output.size(0), encoder_output.size(2))).unsqueeze(1).to(model_config.device)

In [291]:
decoder_output, decoder_hidden, context, weights = decoder(input_var, encoder_hidden, encoder_output, context)

In [16]:
def train_model(model, optimizer, train_loader, criterion):
    model.train()
    loss_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch['label'])
        loss.backward()
        optimizer.step()
        loss_train += loss.item() * \
            len(batch['label']) / len(train_loader.dataset)
    return loss_train

In [17]:
optimizer = torch.optim.Adam(translation_model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
train_model(translation_model, optimizer, train_loader, criterion)

KeyError: 'label'